In [3]:
import scanpy as sc
import scanorama
import os
import numpy as np
from matplotlib.pyplot import rc_context
import pandas as pd
from glob import glob
from collections import defaultdict

In [4]:
# Define the directory path
directory = '/Users/lidiayung/PhD_project/project_UCD_blca/blca_DATA/blca_DATA_mouse_GSE174182_RAW'
# Get the list of files in the directory (non-recursive)
dirs = os.listdir(directory)

# Create an empty list to store the names
names_list = []

# Extract the unique names from the first 20 characters of the filenames
for x in dirs:
    name = x[:20]
    names_list.append(name)

# Remove duplicates by converting to a set and then back to a list
names_list = list(set(names_list))

# Print the unique names
print(names_list)


['GSM5288670_Sample-5_', 'GSM5288674_Sample-11', 'GSM5288673_Sample-8_', '.DS_Store', 'GSM5288668_Sample-3_', '.Rhistory', 'GSM5288671_Sample-6_', 'GSM5288672_Sample-7_', 'GSM5288669_Sample-4_']


In [5]:
os.chdir(directory)

In [8]:
names_list=['GSM5288669_Sample-4_', 'GSM5288670_Sample-5_', 'GSM5288671_Sample-6_', 
            'GSM5288668_Sample-3_', 'GSM5288672_Sample-7_', 'GSM5288673_Sample-8_', 'GSM5288674_Sample-11_']

In [29]:
from scipy.sparse import csr_matrix


In [30]:
adata_list = []

# Loop over each sample and read in the AnnData object
for name in names_list:

    mtx =f"{name}filtered_matrix.mtx.gz"
    adata = sc.read_mtx(mtx)
    cells=pd.read_csv(f'{name}filtered_barcodes.tsv.gz',header=None)
    features=pd.read_csv(f'{name}filtered_features.tsv.gz',header=None,sep='\t')
    adata= adata.T
    adata.X = csr_matrix(adata.X)

    #check the columns first to make sure they are the ones you need 
    adata.obs['CellID']= cells[0].tolist()
    adata.var['Gene']= features[1].tolist()
    adata.var.index= adata.var['Gene']
    adata.var_names_make_unique() 

    sc.pp.filter_cells(adata, min_genes=300)
    sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
    upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .97)
    adata = adata[adata.obs.n_genes_by_counts < upper_lim]

    adata.obs['source'] = name[:10]
      
    sc.pp.normalize_total(adata, inplace=True)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000, inplace=True)
    
    adata_list.append(adata)

/var/folders/qq/n8l4bgxs53j_s3twq5jzc7g00000gn/T/ipykernel_98321/3371234420.py:24: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['source'] = name[:10]
/Users/lidiayung/miniconda3/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:220: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/var/folders/qq/n8l4bgxs53j_s3twq5jzc7g00000gn/T/ipykernel_98321/3371234420.py:24: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['source'] = name[:10]
/Users/lidiayung/miniconda3/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:220: FutureWarning: The default of observed=False is deprecate

In [31]:
scanorama.integrate_scanpy(adata)

Found 31053 genes among all datasets
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Processing datasets (6345, 6357)
Processing datasets (6329, 6353)
Processing datasets (6322, 6350)
Processing datasets (6321, 6324)
Processing datasets (6312, 6315)
Processing datasets (6311, 6353)
Processing datasets (6303, 6304)
Processing datasets (6295, 6361)
Processing datasets (6286, 6331)
Processing datasets (6286, 6288)
Processing datasets (6267, 6357)
Processing datasets (6266, 6291)
Processing datasets (6260, 6332)
Processing datasets (6260, 6266)
Processing datasets (6257, 6325)
Processing datasets (6257, 6317)
Processing datasets (6255, 6357)
Processing datasets (6255, 6345)
Processing datasets (6255, 6267)
Processing datasets (6253, 6353)
Processing datasets (6253, 6311)
Processing datasets (6251, 6269)
Processing datasets (6250, 6306)
Processing datasets (6250, 6272)
Processing data

/Users/lidiayung/miniconda3/lib/python3.9/site-packages/scanorama/scanorama.py:284: ImplicitModificationWarning: Setting element `.obsm['X_scanorama']` of view, initializing view as actual.
  adata.obsm['X_scanorama'] = X_dimred


In [ ]:
adata_scanorama = sc.concat(adata, index_unique='_')

In [14]:
adata.layers['counts'] = adata.X.copy()

In [17]:
adata

AnnData object with n_obs × n_vars = 36337 × 31053
    obs: 'CellID', 'n_genes', 'n_genes_by_counts', 'total_counts', 'source'
    var: 'Gene', 'n_cells_by_counts-GSM5288668', 'mean_counts-GSM5288668', 'pct_dropout_by_counts-GSM5288668', 'total_counts-GSM5288668', 'n_cells_by_counts-GSM5288669', 'mean_counts-GSM5288669', 'pct_dropout_by_counts-GSM5288669', 'total_counts-GSM5288669', 'n_cells_by_counts-GSM5288670', 'mean_counts-GSM5288670', 'pct_dropout_by_counts-GSM5288670', 'total_counts-GSM5288670', 'n_cells_by_counts-GSM5288671', 'mean_counts-GSM5288671', 'pct_dropout_by_counts-GSM5288671', 'total_counts-GSM5288671', 'n_cells_by_counts-GSM5288672', 'mean_counts-GSM5288672', 'pct_dropout_by_counts-GSM5288672', 'total_counts-GSM5288672', 'n_cells_by_counts-GSM5288673', 'mean_counts-GSM5288673', 'pct_dropout_by_counts-GSM5288673', 'total_counts-GSM5288673'
    layers: 'counts'

In [19]:
import scvi

In [ ]:

scvi.model.SCVI.setup_anndata(adata, layer = "counts",
                             categorical_covariate_keys=["source"],
                             continuous_covariate_keys=['total_counts'])


model = scvi.model.SCVI(adata)